In [1]:
import copy
from pathlib import Path
import warnings
import random

import numpy as np
import pandas as pd
import pytorch_lightning as pl
from pytorch_lightning.callbacks import EarlyStopping, LearningRateMonitor
from pytorch_lightning.loggers import TensorBoardLogger
import torch

import matplotlib.pyplot as plt

from pytorch_forecasting import Baseline, TemporalFusionTransformer, TimeSeriesDataSet
from pytorch_forecasting.data import GroupNormalizer
from pytorch_forecasting.metrics import SMAPE, PoissonLoss, QuantileLoss
from pytorch_forecasting.models.temporal_fusion_transformer.tuning import optimize_hyperparameters

In [2]:
input_length = 168
output_length = 24

show_graph = False

In [3]:
# model_paths = ["Save_File/experiment/A7/4108.ckpt",
#                "Save_File/experiment/A7/A7.ckpt"]

# building_num = [41,41]

model_paths = ["Save_File/experiment/A3/A3.ckpt",
               "Save_File/experiment/A3/3401.ckpt",
               "Save_File/experiment/A3/3402.ckpt",
               "Save_File/experiment/A3/3403.ckpt",
               "Save_File/experiment/A3/3404.ckpt",
               "Save_File/experiment/A3/3405.ckpt",
               "Save_File/experiment/A3/3406.ckpt",
               "Save_File/experiment/A3/3407.ckpt",
               "Save_File/experiment/A3/3408.ckpt",
               "Save_File/experiment/A3/3409.ckpt",
               "Save_File/experiment/A3/3410.ckpt"]

building_num = [34 for i in range(12)]

data_path = "./Refined_Data/Grouped_Data/Input_Data2.csv"

In [4]:
dataframes = pd.read_csv(data_path, parse_dates = ["date_time"])

dataframes['num']     =   dataframes['num'].apply(str)
dataframes['Week']    =   dataframes['Week'].apply(str)
dataframes['24Hour']  =   dataframes['24Hour'].apply(str)
dataframes['holiday'] =   dataframes['holiday'].apply(str)
dataframes['Weekend'] =   dataframes['Weekend'].apply(str)
dataframes['energy_group'] = dataframes['energy_group'].apply(str)

# random_choice = random.randrange(2,13)

# start_point = -168 * random_choice - 1
# duration = 168

def smape(A, F):
    return 100/len(A) * np.sum(2 * np.abs(F - A) / (np.abs(A) + np.abs(F)))

all_smape = [[] for i in range(len(model_paths))]
step_smape = [[] for i in range(2, 13)]
step_mean_smape = []

In [5]:
for random_choice in range(2, 13):

    print(f"selected time slice : {random_choice}")
    start_point = -168 * random_choice - 1
    duration = 168

    for i in range(len(model_paths)):
        now_building_dataframe = dataframes.loc[dataframes["num"] == str(building_num[i])].copy()
        now_building_dataframe = now_building_dataframe[start_point:start_point+input_length+duration]
        
        now_best_tft = TemporalFusionTransformer.load_from_checkpoint(model_paths[i])

        original = now_building_dataframe.iloc[0:input_length]["kWH"].tolist()
        prediction = now_building_dataframe.iloc[0:input_length]["kWH"].tolist()  

        total_cycle = ((len(now_building_dataframe) - input_length - output_length) // output_length) + 1

        for k in range(total_cycle):
            now_start_point = k*output_length

            encoder_data = now_building_dataframe.iloc[now_start_point:now_start_point+input_length].copy()
            decoder_data = now_building_dataframe.iloc[now_start_point+input_length:now_start_point+input_length+output_length].copy()
            new_prediction_data = pd.concat([encoder_data, decoder_data], ignore_index=True)

            raw_predictions = now_best_tft.predict(new_prediction_data, mode="prediction").numpy().tolist()[0]
            originals = now_building_dataframe.iloc[now_start_point+input_length : now_start_point+input_length+output_length]["kWH"].tolist()

            prediction.extend(raw_predictions)
            original.extend(originals)

            # for next step, change dataframe's original value to predicted value
            for p in range(output_length):
                now_building_dataframe.iloc[now_start_point+input_length+p, now_building_dataframe.columns.get_loc("kWH")] = raw_predictions[p]
            
        if show_graph:
            plt.rcParams["figure.figsize"] = (17,5)
            fig = plt.figure()
            graph = fig.add_subplot(1, 1, 1)
            graph.plot(prediction, color='blue')
            graph.plot(original, color='red')
            plt.show()

        smape_loss = smape(np.array(prediction[168:]), np.array(original[168:]))
        print(f"building : {model_paths[i]}, score : {smape_loss}")
        all_smape[i].append(smape_loss)
        step_smape[random_choice-2].append(smape_loss)
    print(f"this slice : {random_choice}, score : {np.mean(np.array(step_smape[random_choice-2]))}")
    step_mean_smape.append(np.mean(np.array(step_smape[random_choice-2])))

selected time slice : 2
building : Save_File/experiment/A3/A3.ckpt, score : 3.3621994936102357
building : Save_File/experiment/A3/3401.ckpt, score : 2.9296241266611007
building : Save_File/experiment/A3/3402.ckpt, score : 12.860689581759575
building : Save_File/experiment/A3/3403.ckpt, score : 8.673952305414286
building : Save_File/experiment/A3/3404.ckpt, score : 5.252470729710511
building : Save_File/experiment/A3/3405.ckpt, score : 10.472833967756557
building : Save_File/experiment/A3/3406.ckpt, score : 13.308562954726707
building : Save_File/experiment/A3/3407.ckpt, score : 8.716889602739919
building : Save_File/experiment/A3/3408.ckpt, score : 7.5379549059503415
building : Save_File/experiment/A3/3409.ckpt, score : 6.65579319878349
building : Save_File/experiment/A3/3410.ckpt, score : 7.086558140430188
this slice : 2, score : 7.896139000685719
selected time slice : 3
building : Save_File/experiment/A3/A3.ckpt, score : 4.574367275075132
building : Save_File/experiment/A3/3401.ckpt,

In [6]:
print(f"total score mean : {np.mean(np.array(step_mean_smape))}")
for i in range(len(all_smape)):
    print(f"index : {i}, totoal score : {np.mean(np.array(all_smape[i]))}, min : {np.min(np.array(all_smape[i]))}, max : {np.max(np.array(all_smape[i]))}")

total score mean : 9.433654036034916
index : 0, totoal score : 4.807811437783955, min : 3.3621994936102357, max : 5.8407160941160905
index : 1, totoal score : 1.98040233767299, min : 1.663695529279958, max : 2.9296241266611007
index : 2, totoal score : 14.435384101121098, min : 7.859278901257335, max : 20.639892432105576
index : 3, totoal score : 8.700836176257809, min : 7.069446195648126, max : 11.138970663005466
index : 4, totoal score : 7.800193404350309, min : 5.252470729710511, max : 9.572944788812368
index : 5, totoal score : 10.715229135427952, min : 8.356852046552232, max : 14.028480433145122
index : 6, totoal score : 17.092579849851532, min : 13.308562954726707, max : 21.99104331368191
index : 7, totoal score : 10.686879947145922, min : 8.285269839693512, max : 13.803947998787871
index : 8, totoal score : 9.377048064852376, min : 7.206518295059643, max : 12.543771332733334
index : 9, totoal score : 7.575852964483034, min : 6.65579319878349, max : 9.606224376775687
index : 10, 